# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

X. Zhang  ->  X. Zhang  |  ['X. Zhang']
O. Krause  ->  O. Krause  |  ['O. Krause']
M. Güdel  ->  M. Güdel  |  ['M. Güdel']
Arxiv has 50 new papers today
          2 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/2 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2307.06596


extracting tarball to tmp_2307.06596...

 done.
Retrieving document from  https://arxiv.org/e-print/2307.06692


extracting tarball to tmp_2307.06692...

 done.


/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3508: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3508: LatexWarning: Found documentclass in tmp_2307.06692/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'dust_temporal_evolution.tex' from 'tmp_2307.06692/dust_temporal_evolution.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'dust_fit_parameters.tex' from 'tmp_2307.06692/dust_fit_parameters.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


O. Krause  ->  O. Krause  |  ['O. Krause']
M. Güdel  ->  M. Güdel  |  ['M. Güdel']


Found 93 bibliographic references in tmp_2307.06692/main.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2307.06692-b31b1b.svg)](https://arxiv.org/abs/arXiv:2307.06692) | **Ejecta, Rings, and Dust in SN 1987A with JWST MIRI/MRS**  |
|| O. C. Jones, et al. -- incl., <mark>O. Krause</mark>, <mark>M. Güdel</mark> |
|*Appeared on*| *2023-07-14*|
|*Comments*| *27 pages, 16 figures, 4 tables. Submitted ApJ*|
|**Abstract**| Supernova (SN) 1987A is the nearest supernova in $\sim$400 years. Using the {\em JWST} MIRI Medium Resolution Spectrograph, we spatially resolved the ejecta, equatorial ring (ER) and outer rings in the mid-infrared 12,927 days after the explosion. The spectra are rich in line and dust continuum emission, both in the ejecta and the ring. Broad emission lines (280-380~km~s$^{-1}$ FWHM) seen from all singly-ionized species originate from the expanding ER, with properties consistent with dense post-shock cooling gas. Narrower emission lines (100-170~km~s$^{-1}$ FWHM) are seen from species originating from a more extended lower-density component whose high ionization may have been produced by shocks progressing through the ER, or by the UV radiation pulse associated with the original supernova event. The asymmetric east-west dust emission in the ER has continued to fade, with constant temperature, signifying a reduction in dust mass. Small grains in the ER are preferentially destroyed, with larger grains from the progenitor surviving the transition from SN into SNR. The ER is fit with a single set of optical constants, eliminating the need for a secondary featureless hot dust component. We find several broad ejecta emission lines from [Ne~{\sc ii}], [Ar~{\sc ii}], [Fe~{\sc ii}], and [Ni~{\sc ii}]. With the exception of [Fe~{\sc ii}]~25.99$\mu$m, these all originate from the ejecta close to the ring and are likely being excited by X-rays from the interaction. The [Fe~{\sc ii}]~5.34$\mu$m to 25.99$\mu$m line ratio indicates a temperature of only a few hundred K in the inner core, consistent with being powered by ${}^{44}$Ti decay. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2307.06596-b31b1b.svg)](https://arxiv.org/abs/arXiv:2307.06596) | **Investigating 16 Open Clusters in the Kepler/K2-Gaia DR3 field. I.  Membership, Binary, and Rotation**  |
|| L. Long, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2023-07-14*|
|*Comments*| *21 pages, 17 figures, accepted for publication in ApJS*|
|**Abstract**| Using data from the Gaia Data Release 3 (Gaia DR3) and Kepler/K2, we present a catalog of 16 open clusters with ages ranging from 4 to 4000 Myr, which provides detailed information on membership, binary systems, and rotation. We assess the memberships in 5D phase space, and estimate the basic parameters of each cluster. Among the 20,160 members, there are 4,381 stars identified as binary candidates and 49 stars as blue straggler stars. The fraction of binaries vary in each cluster, and the range between 9% to 44%. We obtain the rotation periods of 5,467 members, of which 4,304 are determined in this work. To establish a benchmark for the rotation-age-color relation, we construct color-period diagrams. We find that the rotational features of binaries are similar to that of single stars, while features for binaries are more scattered in the rotation period. Moreover, the morphology of the color-period relationship is already established for Upper Scorpius at the age of 19 Myr, and some stars of varying spectral types (i.e. FG-, K-, and M-type) show different spin-down rates after the age of ~110 Myr. By incorporating the effects of stalled spin-down into our analysis, we develop an empirical rotation-age-color relation, which is valid with ages between 700 - 4000 Myr and colors corresponding to a range of 0.5 < (G_BP-G_RP)0 < 2.5 mag. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2307.06692.md
    + _build/html/tmp_2307.06692/./Figures/eq_extraction_regions.png
    + _build/html/tmp_2307.06692/./Figures/cps_extraction_regions.png
    + _build/html/tmp_2307.06692/./Figures/day12927_new_fit_astrodust.png
    + _build/html/tmp_2307.06692/./Figures/day12927_new_fit_crb_sil.png
    + _build/html/tmp_2307.06692/./Figures/day12927_new_fit_fe_sil.png
    + _build/html/tmp_2307.06692/./Figures/cubes_det_features.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\Msun}{ M_\odot  }$
$\newcommand{\dg}{^\circ}$
$\newcommand{\kms}{\rm{km s^{-1}}}$
$\newcommand{\ergcms}{\rm erg~cm^{-2}~s^{-1}}$</div>



<div id="title">

# Ejecta, Rings, and Dust in SN 1987A with JWST MIRI/MRS

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2307.06692-b31b1b.svg)](https://arxiv.org/abs/2307.06692)<mark>Appeared on: 2023-07-14</mark> -  _27 pages, 16 figures, 4 tables. Submitted ApJ_

</div>
<div id="authors">

O. C. Jones, et al. -- incl., <mark>O. Krause</mark>, <mark>M. Güdel</mark>

</div>
<div id="abstract">

**Abstract:** Supernova (SN) 1987A is the nearest supernova in $\sim$ 400 years. Using the $* JWST*$ MIRI Medium Resolution Spectrograph, we spatially resolved the ejecta, equatorial ring (ER) and outer rings in the mid-infrared 12,927 days after the explosion.The spectra are rich in line and dust continuum emission, both in the ejecta and the ring.Broad emission lines (280-380 km s $^{-1}$ FWHM) seen from all singly-ionized species originate from the expanding ER, with properties consistent with dense post-shock cooling gas.Narrower emission lines (100-170 km s $^{-1}$ FWHM) are seen from species originating from a more extended lower-density component whose high ionization may have been produced by shocks progressing through the ER, or by the UV radiation pulse associated with the original supernova event.The asymmetric east-west dust emission in the ER has continued to fade, with constant temperature, signifying a reduction in dust mass. Small grains in the ER are preferentially destroyed, with larger grains from the progenitor surviving the transition from SN into SNR.The ER is fit with a single set of optical constants, eliminating the need for a secondary featureless hot dust component.We find several broad ejecta emission lines from [ Ne ${\sc ii}$ ] , [ Ar ${\sc ii}$ ] , [ Fe ${\sc ii}$ ] , and [ Ni ${\sc ii}$ ] . With the exception of [ Fe ${\sc ii}$ ] 25.99 $\mu$ m, these all originate from the ejecta close to the ring and are likely being excited by X-rays from the interaction. The [ Fe ${\sc ii}$ ] 5.34 $\mu$ m to 25.99 $\mu$ m line ratio indicates a temperature of only a few hundred K in the inner core, consistent with being powered by $ ^{44}$ Ti decay.

</div>

<div id="div_fig1">

<img src="tmp_2307.06692/./Figures/eq_extraction_regions.png" alt="Fig4.1" width="50%"/><img src="tmp_2307.06692/./Figures/cps_extraction_regions.png" alt="Fig4.2" width="50%"/>

**Figure 4. -** Top row: The spectra extraction regions for the ER spectra plotted on band A in each channel. The total ER spectrum was extracted from the white elliptical annulus, with the east and west ER extracted from the segments on the left and right. Bottom row: Same as top but for the cardinal point and ejecta spectra. In both rows, the location of the outer rings is shown by the two faded ellipses, the channels 1--3 background inner boundaries are shown by the dashed cyan circles, and the channel 4 background region is shown by the dashed cyan polygon. The `entire' SN 1987A system spectrum was extracted from a region slightly smaller than the inner boundary of the background regions and excluding Star 3. North is up, east is left. (*Fig:spec_extraction_Regions*)

</div>
<div id="div_fig2">

<img src="tmp_2307.06692/./Figures/day12927_new_fit_astrodust.png" alt="Fig11.1" width="33%"/><img src="tmp_2307.06692/./Figures/day12927_new_fit_crb_sil.png" alt="Fig11.2" width="33%"/><img src="tmp_2307.06692/./Figures/day12927_new_fit_fe_sil.png" alt="Fig11.3" width="33%"/>

**Figure 11. -** Fits to the MRS and NIRspec data using the two-temperature dust compositions of _astrodust_ ([Hensley and Draine 2023]())  in the top panel, carbon and silicates  ([Draine and Lee 1984]())  in the middle panel, and silicates  ([Draine and Lee 1984]())  and iron dust in the bottom panel. The synchrotron emission (orange line) has been extrapolated from ALMA observations based on the models described in [Cendes, Gaensler and Ng (2018)]() and [Cigan, Matsuura and Gomez (2019)]()(see text for details). The emission from the H and He continuum and lines (purple line) is described in [Larsson, Fransson and Sargent (2023)]().  The best-fit parameters for the dust components are summarized in Table \ref{Table:dusttab}. (*Fig:astrodust_fit*)

</div>
<div id="div_fig3">

<img src="tmp_2307.06692/./Figures/cubes_det_features.png" alt="Fig3" width="100%"/>

**Figure 3. -** Sample slices from our 12 MRS sub-band cubes, with the band labels shown in the top-left. The white lines in each pane represent 1.0 arcsec to highlight the increasing size of the FOV, as well as the decreasing spatial resolution from channels 1 to 4. `Star 3' is visible to the lower left of the ER in bands 1A--2A. North is up, east is left. (*Fig:cube_det_feat*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2307.06692"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

228  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

2  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
